In [39]:
import pandas as pd
from prophet import Prophet 
import os
import logging

logging.getLogger('prophet').setLevel(logging.WARNING)

In [40]:
# read the data
data_import = pd.read_csv('Data - VOO.csv', parse_dates=['Date'])
data_import

,Date,Open,High,Low,Close,Adj Close,Volume
0,2010-09-09,102.500000,102.500000,101.139999,101.320000,79.681915,26500
1,2010-09-10,101.680000,101.860001,101.300003,101.779999,80.043694,8600
2,2010-09-13,102.959999,103.139999,102.500000,103.059998,81.050354,33750
3,2010-09-14,102.839996,103.480003,102.379997,103.040001,81.034645,59400
4,2010-09-15,102.620003,103.379997,102.400002,103.300003,81.239082,9250
...,...,...,...,...,...,...,...
3159,2023-03-29,367.399994,368.940002,366.290009,368.690002,368.690002,3636100
3160,2023-03-30,371.230011,371.480011,369.100006,370.890015,370.890015,3316700
3161,2023-03-31,371.779999,376.339996,371.649994,376.070007,376.070007,4157100
3162,2023-04-03,375.600006,377.910004,375.220001,377.559998,377.559998,3525300


In [41]:
# Add column of the day names

day_mapper = {0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3:'Thursday', 4:'Friday', 5:'Saturday', 6:'Sunday'}
data_import['DayOfWeek'] = data_import['Date'].map(lambda x: day_mapper[x.dayofweek])
data_import

,Date,Open,High,Low,Close,Adj Close,Volume,DayOfWeek
0,2010-09-09,102.500000,102.500000,101.139999,101.320000,79.681915,26500,Thursday
1,2010-09-10,101.680000,101.860001,101.300003,101.779999,80.043694,8600,Friday
2,2010-09-13,102.959999,103.139999,102.500000,103.059998,81.050354,33750,Monday
3,2010-09-14,102.839996,103.480003,102.379997,103.040001,81.034645,59400,Tuesday
4,2010-09-15,102.620003,103.379997,102.400002,103.300003,81.239082,9250,Wednesday
...,...,...,...,...,...,...,...,...
3159,2023-03-29,367.399994,368.940002,366.290009,368.690002,368.690002,3636100,Wednesday
3160,2023-03-30,371.230011,371.480011,369.100006,370.890015,370.890015,3316700,Thursday
3161,2023-03-31,371.779999,376.339996,371.649994,376.070007,376.070007,4157100,Friday
3162,2023-04-03,375.600006,377.910004,375.220001,377.559998,377.559998,3525300,Monday


# Method 1: Average Dip per Day

In [42]:
# Take the previous value and then subtract it from the current value, check the difference

data_import['diff_from_previous_day'] = (data_import['Open'].diff() / data_import['Open']) * 100 # .diff() takes the value of the current row - previous row
data_import

,Date,Open,High,Low,Close,Adj Close,Volume,DayOfWeek,diff_from_previous_day
0,2010-09-09,102.500000,102.500000,101.139999,101.320000,79.681915,26500,Thursday,NaN
1,2010-09-10,101.680000,101.860001,101.300003,101.779999,80.043694,8600,Friday,-0.806452
2,2010-09-13,102.959999,103.139999,102.500000,103.059998,81.050354,33750,Monday,1.243200
3,2010-09-14,102.839996,103.480003,102.379997,103.040001,81.034645,59400,Tuesday,-0.116689
4,2010-09-15,102.620003,103.379997,102.400002,103.300003,81.239082,9250,Wednesday,-0.214376
...,...,...,...,...,...,...,...,...,...
3159,2023-03-29,367.399994,368.940002,366.290009,368.690002,368.690002,3636100,Wednesday,1.050622
3160,2023-03-30,371.230011,371.480011,369.100006,370.890015,370.890015,3316700,Thursday,1.031710
3161,2023-03-31,371.779999,376.339996,371.649994,376.070007,376.070007,4157100,Friday,0.147934
3162,2023-04-03,375.600006,377.910004,375.220001,377.559998,377.559998,3525300,Monday,1.017041


In [43]:
# Compare the different days to see which one has the biggest dip from the previous day

data_import.groupby('DayOfWeek')['diff_from_previous_day'].mean()

DayOfWeek
Friday       0.063778
Monday      -0.043438
Thursday    -0.011719
Tuesday      0.111333
Wednesday    0.051330
Name: diff_from_previous_day, dtype: float64

This supports the weekend effect that is know in finance: stock returns on Mondays are often significantly lower than those of the immediately preceding Friday.

# Method 2: Backtester Method

Tracks the number of shares that I own based on the number of shares that I bought when and what happened to the price for any given day

In [44]:
# Create a table of every day between the minimum date to deal with holidays
dates = pd.date_range(start = data_import['Date'].min(), end = data_import['Date'].max())

# Add the name of the day of the week
date_table = pd.DataFrame(data = {'Calendar Date':dates})
date_table['Weekday'] = date_table['Calendar Date'].map(lambda x: day_mapper[x.dayofweek])
date_table

,Calendar Date,Weekday
0,2010-09-09,Thursday
1,2010-09-10,Friday
2,2010-09-11,Saturday
3,2010-09-12,Sunday
4,2010-09-13,Monday
...,...,...
4586,2023-03-31,Friday
4587,2023-04-01,Saturday
4588,2023-04-02,Sunday
4589,2023-04-03,Monday


In [45]:
# Create a table of all dates and remove the weekends

full_calendar = pd.merge(left = date_table, right = data_import, how = 'left', left_on='Calendar Date', right_on='Date')
full_calendar = full_calendar[~full_calendar['Weekday'].isin(['Saturday', 'Sunday'])]
full_calendar.rename(columns = {'Date': 'Trading Day'}, inplace = True )
full_calendar

,Calendar Date,Weekday,Trading Day,Open,High,Low,Close,Adj Close,Volume,DayOfWeek,diff_from_previous_day
0,2010-09-09,Thursday,2010-09-09,102.500000,102.500000,101.139999,101.320000,79.681915,26500.0,Thursday,NaN
1,2010-09-10,Friday,2010-09-10,101.680000,101.860001,101.300003,101.779999,80.043694,8600.0,Friday,-0.806452
4,2010-09-13,Monday,2010-09-13,102.959999,103.139999,102.500000,103.059998,81.050354,33750.0,Monday,1.243200
5,2010-09-14,Tuesday,2010-09-14,102.839996,103.480003,102.379997,103.040001,81.034645,59400.0,Tuesday,-0.116689
6,2010-09-15,Wednesday,2010-09-15,102.620003,103.379997,102.400002,103.300003,81.239082,9250.0,Wednesday,-0.214376
...,...,...,...,...,...,...,...,...,...,...,...
4584,2023-03-29,Wednesday,2023-03-29,367.399994,368.940002,366.290009,368.690002,368.690002,3636100.0,Wednesday,1.050622
4585,2023-03-30,Thursday,2023-03-30,371.230011,371.480011,369.100006,370.890015,370.890015,3316700.0,Thursday,1.031710
4586,2023-03-31,Friday,2023-03-31,371.779999,376.339996,371.649994,376.070007,376.070007,4157100.0,Friday,0.147934
4589,2023-04-03,Monday,2023-04-03,375.600006,377.910004,375.220001,377.559998,377.559998,3525300.0,Monday,1.017041


In [46]:
# In order to account for holidays, take the next day's opening value and that's the price I'd buy the stock at
full_calendar = full_calendar.bfill(axis = 'rows').reset_index()

In [47]:
# There's a different number of each Weekday across 12 years
full_calendar['Weekday'].value_counts()


Thursday     656
Friday       656
Tuesday      656
Monday       656
Wednesday    655
Name: Weekday, dtype: int64

In [48]:
# For each weekday from 'Monday' to 'Friday' we need to create a function to backtest as if we invested from the beginning

def day_backtester(day, amount_to_invest, data):
    temp_data = data[data['Weekday'] == day]
    temp_data['Shares Owned'] = amount_to_invest / temp_data['Open']
    final_price = temp_data['Open'].iloc[-1]
    final_amount = temp_data['Shares Owned'].sum() * final_price
    formatted_final_amount = "${:,.2f}".format(final_amount)
    return formatted_final_amount

In [50]:
for i in full_calendar['Weekday'].unique():
    print(i, day_backtester(i, 100 ,full_calendar))

Thursday $123,710.46
Friday $123,818.94
Monday $125,095.36
Tuesday $125,872.62
Wednesday $122,084.81


/Users/jeffreysedlock/miniconda3/envs/time_series/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


As we can see above, investing on Tuesday of every week would have yieled the highest returns the past 13 years. This disproves the weekend effect for our purposes.